In [1]:
# find output/ -type f -name "*.csv" -newermt 2020-09-17 -not -path "*/.virtfs_metadata/*" | xargs wc -l
import os
import os.path
from os import path
import pandas as pd
from stat import S_ISREG, ST_CTIME, ST_MODE
import time
from datetime import datetime
import re

 
dfs = []
rootDir = 'output2'
for dirName, subdirList, fileList in os.walk(rootDir, topdown=False):
    if ".virtfs_metadata" in dirName:
        continue
    for fname in fileList:
        # print('%s\t%s' % (dirName, fname))
        stat_file = os.stat(os.path.join(dirName, fname))
        cdate = stat_file[ST_CTIME]
        creationtime_file = time.ctime(os.path.getmtime(os.path.join(dirName, fname)))
        creationdate_file = datetime.fromtimestamp(os.path.getmtime(os.path.join(dirName, fname))).date()
        targeted_date = datetime.strptime("2020-09-16", "%Y-%m-%d")
        # if creationdate_file.day != targeted_date.day:
        #    continue
        if fname.endswith(".csv"):
            # print(dirName)
            def is_file_empty(file_path):
                """ Check if file is empty by confirming if its size is 0 bytes"""
                # Check if file exist and it is empty
                return os.path.exists(file_path) and os.stat(file_path).st_size == 0
            if is_file_empty(os.path.join(dirName, fname)):
                # print("empty CSV file")
                continue
            df = pd.read_csv(os.path.join(dirName, fname))
            df['sampler'] = fname
            
            
            # processing metainformation (README)
            default_sampler = 9 # KUS is the default sampler
            if path.isfile(os.path.join(dirName, 'README.md')):
                with open(os.path.join(dirName, 'README.md'), 'r') as readme:
                    content_readme = readme.read()
                    # print(content_readme)
                    m = re.search('ref_sampler=(\d+)', content_readme)
                    if m is not None:
                        default_sampler = int(m.group(1))
                    else:
                        # print("default sampler is used... KUS", default_sampler)
                        continue
            df['ref_sampler'] = default_sampler
            dfs.append(df)

# for df in dfs:
    # df['sampler'] = solver
#    if len(df['file']) == 1 and "FM-3" in df['file'][0]:
#        print(solver)
#        print(df['file'])
#        print(df['Uniform'].value_counts(dropna=False))
#        print()


all_results = pd.concat(dfs)

all_results.to_csv('all-results-sampler.csv')
all_results[['file', 'time', 'Uniform', 'Timeout', 'sampler']].to_csv('all-results-sampler-digest.csv')

In [2]:
all_results.drop_duplicates(subset=['file', 'sampler', 'Uniform', 'Timeout'], inplace=True)
# all_results.query("sampler == 'Uniform-DistanceBasedSampling.csv'")

i = 0
df_summary = pd.DataFrame(columns=['sampler', "nbench", "nTrueUniform", "nFalseUniform", "nNAUniform"])
for s in all_results["sampler"].unique():
    nbench = len(all_results.query("sampler == '" + s + "'"))
    ntrueu = len(all_results.query("Uniform == True and sampler == '" + s + "'"))
    nfalseu = len(all_results.query("Uniform == False and sampler == '" + s + "'"))
    nnau = len(all_results.query("Uniform != True and Uniform != False and sampler == '" + s + "'"))
    print(s, nbench, ntrueu)
    df_summary.loc[i] = (s, nbench, ntrueu, nfalseu, nnau)
    i = i + 1

df_summary.to_csv('summary-uniformity-samplers.csv')
df_summary
#all_results.query("sampler == 'Uniform-SPUR.csv' and Uniform == False")

Uniform-DistanceBasedSampling.csv 1 0
Uniform-SPUR.csv 18 7
Uniform-QuickSampler.csv 1 0
Uniform-SMARCH.csv 1 0
Uniform-CustomSampler.csv 1 1
Uniform-KUS.csv 1 1
Uniform-STS.csv 1 0


,sampler,nbench,nTrueUniform,nFalseUniform,nNAUniform
0,Uniform-DistanceBasedSampling.csv,1,0,1,0
1,Uniform-SPUR.csv,18,7,2,9
2,Uniform-QuickSampler.csv,1,0,1,0
3,Uniform-SMARCH.csv,1,0,1,0
4,Uniform-CustomSampler.csv,1,1,0,0
5,Uniform-KUS.csv,1,1,0,0
6,Uniform-STS.csv,1,0,0,1


In [3]:
all_results.query("sampler == 'Uniform-KUS.csv' and Uniform == False")['file'].values #[['file', 'time', 'Uniform', 'Timeout', 'sampler']]

array([], dtype=object)

In [4]:
all_results.query("sampler == 'Uniform-SPUR.csv' and Uniform == False")['file'].values #[['file', 'time', 'Uniform', 'Timeout', 'sampler']]

array(['/home/samplingfm/Benchmarks/V7/s27_new_7_4.cnf',
       '/home/samplingfm/Benchmarks/V7/s27_7_4.cnf'], dtype=object)

In [5]:
all_results.query('ref_sampler == ' + str(6))

,file,time,cmd_output,err_output,Uniform,Timeout,sampler,ref_sampler
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,6.560,b'command: log /tmp/output.txt\ncommand: log /...,b'',False,False,Uniform-DistanceBasedSampling.csv,6
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,1.706,b'Experiment: 0 of 1\nconstantFactor:3...,b'',False,False,Uniform-QuickSampler.csv,6
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,1306.412,b'Experiment: 0 of 1\nconstantFactor:3...,b'',False,False,Uniform-SMARCH.csv,6
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,1193.054,b'Experiment: 0 of 1\nconstantFactor:3...,b'',True,False,Uniform-CustomSampler.csv,6
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,1850.024,b'Experiment: 0 of 1\nconstantFactor:3...,b'',True,False,Uniform-KUS.csv,6
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,0.807,b'Experiment: 0 of 1\nconstantFactor:3...,"b'Traceback (most recent call last):\n File ""...",NaN,False,Uniform-STS.csv,6
